## basic config

In [1]:
import torch
torch.cuda.device_count()

1

In [2]:

def align_faces_2(root_path):
    import os 
    current_path = os.getcwd()
    temp_path = "/home/bean/DragVideo/stylegan3-editing"
    os.chdir(temp_path)
    command =f"""python prepare_data/preparing_faces_parallel.py \
                --mode align \
                --root_path  {root_path} 
                """
    import subprocess
    subprocess.run(command, shell=True)
    os.chdir(current_path)

In [3]:
# DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"
DRAGVIDEO_ROOT_PATH = "/home/bean/"

# keep matching encoder,SG as pairt 
old = {"encoder":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
       "SG": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
       "model_name":"stylegan2",
       "resolution":1024,
       }

new = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl",
         "model_name":"stylegan3",
               "resolution":1024,
         }

new_256 = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/Stylegan3_1_files/stylegan3-r-ffhqu-256x256.pkl",
         "model_name":"stylegan3",
         "resolution":256,
         }

videos = {
       "obama": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/obama.mp4",
       "man_speaking": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
       "rahul": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/rahul.mp4",
       "alien_girl": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/alien_girl.mp4",
       "mirrAR": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/mirrAR.mp4",
       "vsauce_frnd":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/micheal_friendzone.mp4",
       
}

In [4]:
EXPERIMENT_NAME_POSTFIX = "vsauce_ffmpeg"#"aligned_sg3_&_new_e4e_reconstruction_invstep_200_pti_step_450"
models_set = new
Testing =True
# --------------------------------  


import datetime
lazy_config = {
    "EXP_NAME": str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+"_"+ EXPERIMENT_NAME_POSTFIX,
    # "EXP_NAME": "/home/bean/DragVideo/Data_store/experiments/_SAVE_mirrAR_200_frames_2",
    "e4e": models_set['encoder'],
    "stylegan2_ada_ffhq": models_set['SG'],
    "video_path":  videos["vsauce_frnd"],
    "model_name" : models_set['model_name'],
    "n_frames" : 1,#120,#200,
    "IMAGE_SIZE": models_set["resolution"],
    "N_STEPS_in_draggan":  "150",
    "editing_function_name":"large_eyes",#"large_eyes" # "make_jaw_wider" # "mouth_wide"
    "fps": 24,
    
}

env_config = {
    "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/" ,
    "init_exp_dir_shell_path": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy",
    
    
}

#hyper parameters in PTI

hyper_config = {
    "max_pti_steps": 350,
    "first_inv_steps":450,
    "max_images_to_invert": 200,
}

if Testing: 
    hyper_config["max_pti_steps"] = 1
    hyper_config["first_inv_steps"] = 1
    hyper_config["max_images_to_invert"] = 5
    lazy_config["N_STEPS_in_draggan"] = '1'
    


# ----------------------------------------------------------------------------------------------

from run_utils import *

# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])



# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
     "quad_values_path": os.path.join(Experiment_path,'quad_values'),
}

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])



# add all these configs to log.txt
# --------------------------------  
with open(os.path.join(Experiment_path,'log.txt'), 'a') as f:
    import json
    f.write(f"lazy_config: {json.dumps(lazy_config, indent=4)}\n")
    f.write(f"env_config: {json.dumps(env_config, indent=4)}\n")
    f.write(f"hyper_config: {json.dumps(hyper_config, indent=4)}\n")
    
    



In [5]:
# extract frames from video
#==============================================================================

from importlib.machinery import SourceFileLoader
# imports the module from the given path
# video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()

ffmpeg_utils = SourceFileLoader("video_utils","../utils_draggan/ffmpeg_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")

# video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])
ffmpeg_utils.ffmpeg.extract_frames(video_path = lazy_config['video_path'],
                                      output_frames_dir = raw_path,
                                      fps = lazy_config['fps'],
                                      first_n_frames = lazy_config['n_frames'],
)

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

# start

In [6]:
# from utils.align_data import pre_process_images


In [7]:
# # # delete this cell 
# # "/home/bean/DragVideo/Data_store/man_speaking/aligned": input_data_path 
# # "/home/bean/DragVideo/Data_store/man_speaking/quad_values", quad_pkl_path

# raw_path = "/home/bean/DragVideo/Data_store/man_speaking/raw"
# pre_process_images(raw_path, IMAGE_SIZE=1024) # o/p: config.input_data_path ,quad pickle dir


In [8]:
from utils.align_data import pre_process_images
pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path ,quad pickle dir
#
# align_faces_2(os.path.join(Experiment_path,'raw'))



100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

saving aligned images...
saving quad values...
done!


In [9]:
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI

from run_utils_2 import load_generators,export_updated_pickle
from configs import paths_config

use_multi_id_training = True
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

generator_type =paths_config.multi_id_model_type if use_multi_id_training else "__"
old_G, new_G = load_generators(model_id, generator_type)
sg_tuned_pkl = export_updated_pickle(new_G,model_id,name = lazy_config["model_name"])

# print(sg_tuned_pkl) # 'QBUXQCXZGWET'



#get landmarks
landmark_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)
print("pwd",os.getcwd())
import sys
sys.path.append(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
torch.Size([3, 1024, 1024]) 3 1024 1024
Setting up PyTorch plugin "bias_act_plugin"... Done.


  0%|          | 0/1 [00:00<?, ?it/s]

Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %106 : int = prim::profile_ivalue(%104)
 does not have profile information (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)
100%|██████████| 1/1 [00:00<00:00, 15.82it/s]


Exporting large updated pickle based off new generator and ffhq.pkl
pwd /home/bean/DragVideo/DragGAN/_facial-landmarks-recognition
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-25_21-36-33_vsauce_ffmpeg/landmarks/000.pkl


Runs draggan part

In [10]:

# just to run this part seperately
import os 
# Experiment_path = "/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_large_eyes_dr_160"
# sg_tuned_pkl = "/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_large_eyes_dr_160/tuned_SG/stylegan3_EZXZEVVUYSPX.pkl"

# Experiment_path =  f"/home/bean/DragVideo/Data_store/experiments/{lazy_config['EXP_NAME']}"	
sg_tuned_pkl = new["SG"]


lazy_config["N_STEPS_in_draggan"]="160"
lazy_config["editing_function_name"]="large_eyes"
lazy_config["IMAGE_SIZE"]="1024"


In [19]:
# clean all data from gpu
import torch
torch.cuda.empty_cache()

os.chdir(env_config["DragGAN_dir"])
import subprocess

print(f"sg_tuned_pkl: {sg_tuned_pkl}")

subprocess.call(['python', '_run_dragvideo.py',
                 '--Experiment_path', Experiment_path,
                 '--N_STEPS',       lazy_config["N_STEPS_in_draggan"],
                 '--CHECKPOINT_PATH', sg_tuned_pkl,
                 "--MAX_SIZE",      str(lazy_config["IMAGE_SIZE"]),
                 "--editing_function_name",lazy_config["editing_function_name"],
])
                #  "--verbose",       "False"])


sg_tuned_pkl: /home/bean/DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl
editing_function_name: large_eyes
intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl"... Done.
()
{'z_dim': 512, 'w_dim': 512, 'channel_base': 65536, 'channel_max': 1024, 'mapping_kwargs': {}, 'conv_kernel': 1, 'filter_size': 6, 'magnitude_ema_beta': 0.9988915792636801, 'output_scale': 0.25, 'c_dim': 0, 'img_resolution': 1024, 'img_channels': 3, 'use_radial_filters': True}
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


  0%|          | 0/160 [00:00<?, ?it/s]/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 51%|█████▏    | 82/160 [01:03<00:59,  1.30it/s]

# post processing

In [12]:
import os
from tqdm import tqdm
os.getcwd()
# import importlib
# importlib.reload(ffmpeg_utils)

'/home/bean/DragVideo/DragGAN'

In [13]:

from utils import de_alignment 
put_back_the_edited_image = de_alignment.put_back_the_edited_image


In [14]:


def paste_edited_faces_back(dir_path,edited_dir='after_drag',save_dir='edit_pasted'):
    raw_dir = os.path.join(dir_path,'raw')
    edited_dir = os.path.join(dir_path,edited_dir)
    quad_dir = os.path.join(dir_path,'quad_values')
    save_dir = os.path.join(dir_path,save_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # get names from edited_dir
    names = [name.split('.')[0] for name in os.listdir(edited_dir)]
    for name in tqdm(names):
        raw_image = os.path.join(raw_dir,name+'.png')
        edited_image = os.path.join(edited_dir,name+'.png')
        quad_path = os.path.join(quad_dir,name+'.pkl')
        save_path = os.path.join(save_dir,name+'.png')
        put_back_the_edited_image(raw_image,edited_image,quad_path,save_path=save_path)


In [15]:
paste_edited_faces_back(Experiment_path,edited_dir="before_drag",save_dir="before_drag_pasted")

0it [00:00, ?it/s]


In [16]:
# dir_path = "/home/bean/DragVideo/Data_store/man_speaking"
paste_edited_faces_back(Experiment_path)

0it [00:00, ?it/s]


In [17]:
# =============================================================================
# using ffmpeg to make video
# =============================================================================
before_drag_pasted_dir = os.path.join(Experiment_path,'before_drag_pasted')
after_drag_pasted_dir = os.path.join(Experiment_path,'edit_pasted')
videos_dir = os.path.join(Experiment_path,'videos')

ffmpeg_utils.ffmpeg.make_video(before_drag_pasted_dir,
                               video_name="pre_drag_full",
                               video_dir=videos_dir,
                               fps=lazy_config['fps'],
                                 )
ffmpeg_utils.ffmpeg.make_video(after_drag_pasted_dir,
                                 video_name="post_drag_full",
                                    video_dir=videos_dir,
                                    fps=lazy_config['fps'],
                                    )


ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

In [18]:
# =============================================================================
# hstack videos
# =============================================================================

video1= os.path.join(videos_dir,"pre_drag_full.mp4")
video2= os.path.join(videos_dir,"post_drag_full.mp4")

# pre_post_drag_full
ffmpeg_utils.ffmpeg.hstack_videos(video1,
                                  video2,
                                  output_dir=videos_dir,
                                  output_name="hstack_pre_post_drag_full",
)

# original_post_drag_full
ffmpeg_utils.ffmpeg.hstack_videos(lazy_config['video_path'],
                                  video2,
                                  output_dir=videos_dir,
                                  output_name="hstack_original_post_drag_full",
)

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

/home/bean/DragVideo/Data_store/experiments/2023-08-25_21-36-33_vsauce_ffmpeg/videos/hstack_pre_post_drag_full.mp4
/home/bean/DragVideo/Data_store/experiments/2023-08-25_21-36-33_vsauce_ffmpeg/videos/hstack_original_post_drag_full.mp4


ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte